# Account, Address 与 Wallet

通过单击顶部栏上的`🚀` -&gt; `Binder`在线运行此示例！

## 准备

这是准备部分，我们准备`web3`实例和`private_key` 。

In [1]:
# prerequisites: 
# we create an account and claim 1000 CFX from the faucet
from pprint import pprint
from conflux_web3 import Web3

w3_ = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))

acct = w3_.account.create()

w3_.cfx.default_account = acct

faucet = w3_.cfx.contract(name="Faucet")
tx_receipt = faucet.functions.claimCfx().transact().executed()

# we use a new w3 object for the following presentation
w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))
private_key: str = acct._private_key.hex()

# 帐户和地址

在 web3 中，通常，拥有一个帐户意味着知道一个**secret**或**private key** 。这个秘密应该保密，没有人知道。

虽然私人应该保密，但***地址***可以透露。地址是从私钥派生的字符串，用于标识帐户。在 Conflux 中，地址以 base32 格式编码。


### `LocalAccount`对象

`w3.account`是一个工厂，用于生成`LocalAccount`对象（例如`random_account` ）。 `LocalAccount`对象包含帐户机密，可用于签署交易。

**注意：不建议手动签署tx，因为它很繁琐，请参阅[钱包](#wallet)部分了解如何使用钱包签署和发送交易。或者您可以参考[construct_transaction_from_scratch](./11-construct_transaction_from_scratch.ipynb)了解如何正确手动签署交易。**

> 更多文档： `w3.account`是一个`cfx_account.Account`对象，继承自`eth_account.Account` ，其大部分api与[eth_account](https://eth-account.readthedocs.io/en/stable/eth_account.html)一致。

In [2]:
random_account = w3.account.from_key(private_key)
print(f"account address: {random_account.address}")
print(f"account private key: {private_key}")

transaction = {
    'to': w3.address.zero_address(),
    'nonce': 1,
    'value': 1,
    'gas': 21000,
    'gasPrice': 10**9,
    'storageLimit': 0,
    'epochHeight': 100,
    'chainId': 1
}
print(f"signed raw tx: {random_account.sign_transaction(transaction).rawTransaction.hex()}")

account address: cfxtest:aam8a8aunke9r62td7c4bef07t21jv02we5kfh832c
account private key: 0xa2f56ba117e4e3b2e3a5feddff642f6020a264573c6d9a7a99d8ad061d7f3109
signed raw tx: 0xf867e301843b9aca00825208940000000000000000000000000000000000000000018064018080a0f6c194255fc2bd426abd6a155022ccde1d9ee5dd35804ec3b95bdf92fa7c7fefa07d4ed5df766ed39c69c95dc879c92d6f9c03351b94cc59e5f4a7117d0012f79b


### Conflux 地址

在 Conflux 中，地址按照[CIP-37](https://github.com/Conflux-Chain/CIPs/blob/master/CIPs/cip-37.md)以 base32 格式编码。您可以简单地从地址文字中推断出该地址属于哪个网络。

In [3]:
# "cfxtest" marks an address in testnet
assert random_account.address.startswith("cfxtest:")

SDK 返回的地址都被类`Base32Address`包裹。该类提供了方便的方法来操作 base32 地址，但您也可以将`Base32Address`对象用作普通的 python `str`对象。您可以访问[Base32Address 文档](https://conflux-fans.github.io/cfx-address/cfx_address.html#module-cfx_address.address)以获取更多信息。

In [4]:
addr = random_account.address
print(f"the type of addr: {type(addr)}")
# a Base32Address object is also a `str`
assert isinstance(addr, str)

# encode a base32 address from hex address and network_id
# it is also supported to use `w3.address("cfxtest:aatp533cg7d0agbd87kz48nj1mpnkca8be1rz695j4")`
address = w3.address("0x1ecde7223747601823f7535d7968ba98b4881e09", network_id=1)
print(address)
pprint([
    address.address_type,
    address.network_id,
    address.hex_address,
    address.verbose_address,
    address.abbr,
    address.mapped_evm_space_address,
    address.eth_checksum_address,
])

the type of addr: <class 'cfx_address.address.Base32Address'>
cfxtest:aatp533cg7d0agbd87kz48nj1mpnkca8be1rz695j4
['user',
 1,
 '0x1ecde7223747601823f7535d7968ba98b4881e09',
 'CFXTEST:TYPE.USER:AATP533CG7D0AGBD87KZ48NJ1MPNKCA8BE1RZ695J4',
 'cfxtest:aat...95j4',
 '0x349f086998cF4a0C5a00b853a0E93239D81A97f6',
 '0x1ECdE7223747601823f7535d7968Ba98b4881E09']


## 钱包

我们使用`wallet`中间件来帮助我们签署交易。如果`w3.wallet`具有`from` address 的帐户，它将对通过`w3.cfx.send_transaction`发送的未签名交易进行签名。

> `wallet`中间件遵循`web3.py`的`construct_sign_and_send_middleware`的实现，但提供了更多功能。例如，我们可以使用`w3.wallet.add_account` 、 `w3.wallet.add_accounts` 、 `w3.wallet.pop`来动态添加或删除帐户。 

In [5]:
# wallet serves as a middleware for conflux_web3 and contains a collection of LocalAccount
assert w3.wallet is w3.middleware_onion["wallet"]

w3.wallet.add_account(random_account)

assert random_account.address in w3.wallet
w3.cfx.send_transaction({
    "from": random_account.address,
    "to": random_account.address,
    "value": 10**18
}).executed()

AttributeDict({'transactionHash': HexBytes('0x909d494560d08badf211b9add10242d1cf2216ef996bc45bc8f54e573a69b26b'),
 'index': 0,
 'blockHash': HexBytes('0xaeca50b3cd33faeedc76e873a75e878bcb18548057961cdc15598a9287e40839'),
 'epochNumber': 98917094,
 'from': 'cfxtest:aam8a8aunke9r62td7c4bef07t21jv02we5kfh832c',
 'to': 'cfxtest:aam8a8aunke9r62td7c4bef07t21jv02we5kfh832c',
 'gasUsed': 21000,
 'gasFee': 21000000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

### 使用`w3.cfx.default_account`将账户添加到钱包

如果设置了`w3.cfx.default_account` ，则将从`w3.cfx.default_account`考虑没有`from`字段的交易。

`w3.cfx.default_account`是一个地址，但您可以使用`LocalAccount`对象进行设置。在这种情况下，提供的帐户将同时添加到钱包中。

```python
w3.cfx.default_account = random_account
```

相当于

```python
w3.cfx.default_account = random_account.address
w3.wallet.add_account(random_account)
```